In [1]:
import pandas as pd
import numpy as np

In [2]:
from src.generate_trees_scores_DEBUG import two_tail_prob

In [3]:
def get_last_detection_id(br):
    return np.array(br)[np.logical_not(np.isnan(br))][-1]

In [27]:
def update_hyps_scores(det, hyps, scores, probz, bg_prob):
    
    detection_id = det.iloc[0].id
    detection_time = det.iloc[0].time
    detection_pos = det.iloc[0].object_location
    
    new_hyps = []
    new_scores = []

    # go through hypotheses
    for hyp_i, hyp in enumerate(hyps):
        existing_score = scores[hyp_i]

        # scenario where the new detection is a new individual
        # pad existing branches

        new_scen = [ h + [np.nan] for h in hyp] + [list(np.empty(len(hyp[0]))*np.nan) + [detection_id]]
        new_hyps.append(new_scen)

        # score is the existing + background prob
        new_scores.append(existing_score + bg_prob)

        # go through the branches in the hypothesis
        for i, branch in enumerate(hyp):

            # scenario where the new detection is a repeat
            repeat_scen = [branch + [detection_id]]
            repeat_scen += [hyp[j] + [np.nan]  for j in range(len(hyp)) if j!=i]

            new_hyps.append(repeat_scen)
            # calculate score
            last_detection = get_last_detection_id(branch)
            new_scores.append( existing_score +probz[int(last_detection), detection_id])
    return new_hyps, new_scores

In [28]:
detections = pd.DataFrame()
detections["id"] = [0, 1, 2]
detections["time"] = [0, 1,2]
detections["object_location"] = [0, 1, 2]

In [29]:
faunastep = 0.8
n_dets = len(detections)
dists = np.zeros((len(detections), len(detections)))
Dtime = np.zeros((len(detections), len(detections)))
for i in range(n_dets):
    for j in range(n_dets):
        dists[i, j] = np.abs(detections.loc[i]["object_location"] - detections.loc[j]["object_location"])
        Dtime[i, j] = np.abs(detections.loc[i]["time"] - detections.loc[j]["time"])

z_stat = np.divide(dists, faunastep * Dtime)
probz = two_tail_prob(z_stat)


/home/nader/Projects/thesis/benthic_targets/venv/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in true_divide
  # Remove the CWD from sys.path while we load stuff.


In [30]:
bg_prob = 0.1

In [31]:
probz

array([[       nan, 0.21129955, 0.21129955],
       [0.21129955,        nan, 0.21129955],
       [0.21129955, 0.21129955,        nan]])

In [32]:
detections

,id,time,object_location
0,0,0,0
1,1,1,1
2,2,2,2


After time 0 we have one hypothesis

In [33]:
hyps = [[[0]]]

In [34]:
scores = []
scores.append(bg_prob)

In [35]:
scores

[0.1]

In [36]:
new_hyps, new_scores = update_hyps_scores(detections[detections.time==1], hyps, scores, probz, bg_prob)

In [37]:
new_hyps, new_scores

([[[0, nan], [nan, 1]], [[0, 1]]], [0.2, 0.3112995473337107])

In [38]:
new_hyps, new_scores = update_hyps_scores(detections[detections.time==2], new_hyps, new_scores, probz, bg_prob)

In [39]:
new_hyps, new_scores

([[[0, nan, nan], [nan, 1, nan], [nan, nan, 2]],
  [[0, nan, 2], [nan, 1, nan]],
  [[nan, 1, 2], [0, nan, nan]],
  [[0, 1, nan], [nan, nan, 2]],
  [[0, 1, 2]]],
 [0.30000000000000004,
  0.4112995473337107,
  0.4112995473337107,
  0.41129954733371066,
  0.5225990946674214])

Get detections at time 1

In [12]:
det_1 = detections[detections.time == 1]

detection_id = det_1.iloc[0].id
detection_time = det_1.iloc[0].time
detection_pos = det_1.iloc[0].object_location


In [13]:
a = [1,2,np.nan,3]
np.array(a)[np.logical_not(np.isnan(a))][-1]

3.0

In [18]:
new_hyps = []
new_scores = []

# go through hypotheses
for hyp_i, hyp in enumerate(hyps):
    existing_score = scores[hyp_i]
    
    # scenario where the new detection is a new individual
    # pad existing branches
  
    new_scen = [ h + [np.nan] for h in hyp] + [list(np.empty(len(hyp[0]))*np.nan) + [detection_id]]
    new_hyps.append(new_scen)
    
    # score is the existing + background prob
    new_scores.append(existing_score + bg_prob)
    
    # go through the branches in the hypothesis
    for i, branch in enumerate(hyp):
       
        # scenario where the new detection is a repeat
        repeat_scen = [branch + [detection_id]]
        repeat_scen += [hyp[j] + [np.nan]  for j in range(len(hyp)) if j!=i]
        
        new_hyps.append(repeat_scen)
        # calculate score
        last_detection = get_last_detection_id(branch)
        new_scores.append( existing_score +probz[int(last_detection), detection_id])
        

In [15]:
new_hyps

[[[0, nan], [nan, 1]], [[0, 1]]]

In [16]:
new_scores

[0.2, 0.3112995473337107]

In [17]:
hyps = new_hyps
scores = new_scores
det_2 = detections[detections["time"]==2]
detection_id = det_2.iloc[0].id
detection_time = det_2.iloc[0].time
detection_pos = det_2.iloc[0].object_location

In [18]:
new_hyps = []
# go through hypotheses
for hyp in hyps:
    
    # scenario where the new detection is a new individual
    # pad existing branches
  
    new_scen = [ h + [np.nan] for h in hyp] + [list(np.empty(len(hyp[0]))*np.nan) + [detection_id]]
    new_hyps.append(new_scen)
    # go through the branches in the hypothesis
    for i, branch in enumerate(hyp):
        # scenario where the new detection is a repeat
        repeat_scen = [branch + [detection_id]]
        repeat_scen += [hyp[j] + [np.nan]  for j in range(len(hyp)) if j!=i]
        
        new_hyps.append(repeat_scen)

In [26]:
for i, h in enumerate(new_hyps):
    print(h, "{:.4f}".format(new_scores[i]))

[[0, nan, nan], [nan, 1, nan], [nan, nan, 2]] 0.3000
[[0, nan, 2], [nan, 1, nan]] 0.4113
[[nan, 1, 2], [0, nan, nan]] 0.4113
[[0, 1, nan], [nan, nan, 2]] 0.4113
[[0, 1, 2]] 0.5226


In [23]:
new_scores

[0.2, 0.3112995473337107]